In [28]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import pandas as pd

import torch.optim as optim
from torchmetrics import Accuracy

In [18]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"

In [19]:
class WaterDataset(Dataset):
    def __init__(self, csv_path):
        super().__init__()
        df = pd.read_csv(csv_path)
        self.data = df.to_numpy()
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, index):
        features = torch.tensor(self.data[index,:-1], dtype=torch.float32)
        label = torch.tensor(self.data[index,-1], dtype=torch.float32)
        return features, label

In [20]:
data_train = WaterDataset("water_potability.csv")

dataloader_train = DataLoader(data_train,
                              batch_size=2,
                              shuffle=True)

dataloader_train

In [21]:
features, label = next(iter(dataloader_train))
print("Features: ",features)
print("Labels: ",label)

Features:  tensor([[7.0173e+00, 1.7013e+02, 2.0169e+04, 5.2163e+00, 3.7823e+02, 5.1277e+02,
         1.5506e+01, 6.4625e+01, 4.8996e+00],
        [       nan, 2.4107e+02, 2.2657e+04, 7.2938e+00,        nan, 3.1560e+02,
         1.6405e+01, 4.6139e+01, 3.7352e+00]])
Labels:  tensor([1., 1.])


Replicating 

 model = nn.Sequential(
    
nn.Linear(9,16),
    
nn.ReLU(),
    
nn.Linear(16,8),
    
nn.ReLU(),
    
nn.Linear(8,1),
    
nn.Sigmoid(),
)

in Class based model.

In [22]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        self.fc1 = nn.Linear(9,16)
        self.fc2 = nn.Linear(16,8)
        self.fc3 = nn.Linear(8,1)
        
    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [23]:
net = Net().to(device=device)
# net.state_dict()

### Training Loop

In [24]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(net.parameters(),lr=0.01)

In [25]:
# Adaptive gradient
adOptimizer = optim.Adagrad(params=net.parameters(), lr= 0.01)

# Adaptive Moment Estimation (Adam)
adam = optim.Adam(net.parameters(), lr=0.01)

In [26]:
epochs = 100

for epoch in range(epochs):
    for features, labels in dataloader_train:
        features = features.to(device)
        labels = labels.to(device)

        optimizer.zero_grad() # clears gradients
        outputs = net(features) # forward pass
        loss = criterion(outputs,labels.view(-1,1)) # compute loss
        loss.backward() # compute gradients
        optimizer.step() # update parameters

In [31]:
acc = Accuracy(task="binary")

net.eval()
with torch.no_grad():
    for features, labels in dataloader_train:
        outputs = net(features)
        preds = (outputs >= 0.5).float()
        acc(preds,labels.view(-1,1))

accuracy = acc.compute()
print(f"Accuracy: {accuracy:.3f}")

Accuracy: 0.610
